# 03 - Parser Benchmark

Comparaison des méthodes d'extraction PDF :
- **pdfplumber** : Parser actuel (extraction de tableaux)
- **Mistral OCR** : Vision model (mistral-ocr-2503)

## Objectif
Mesurer la précision d'extraction des bulletins scolaires vs ground truth.

In [1]:
# ruff: noqa: E402
import json
import os
import sys
import time
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv(project_root / ".env")

# Paths
DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"
GROUND_TRUTH_PATH = DATA_DIR / "ground_truth" / "chiron_ground_truth.json"

print(f"Project root: {project_root}")
print(f"PDFs: {list(RAW_DIR.glob('*.pdf'))}")

Project root: c:\Users\Florent\Documents\data_science\chiron
PDFs: [WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_A.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_B.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_C.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_D.pdf')]


In [2]:
# Charger ground truth
with open(GROUND_TRUTH_PATH, encoding="utf-8") as f:
    ground_truth = json.load(f)

# Index par eleve_id pour comparaison facile
gt_by_eleve = {e["eleve_id"]: e for e in ground_truth["eleves"]}
print(f"Ground truth: {list(gt_by_eleve.keys())}")

Ground truth: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']


## 1. Parser pdfplumber (actuel)

In [3]:
from src.document.bulletin_parser import BulletinParser

parser = BulletinParser()

pdfplumber_results = {}
pdfplumber_times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem  # ELEVE_A, ELEVE_B, etc.
    
    start = time.perf_counter()
    try:
        eleves = parser.parse(pdf_path)
        pdfplumber_results[eleve_id] = eleves[0] if eleves else None
    except Exception as e:
        pdfplumber_results[eleve_id] = f"ERROR: {e}"
    pdfplumber_times[eleve_id] = time.perf_counter() - start
    
    print(f"{eleve_id}: {pdfplumber_times[eleve_id]:.2f}s")

ELEVE_A: 0.07s
ELEVE_B: 0.07s
ELEVE_C: 0.06s
ELEVE_D: 0.07s


In [4]:
# Afficher un exemple de résultat pdfplumber
exemple = pdfplumber_results.get("ELEVE_A")
if exemple and not isinstance(exemple, str):
    print(f"Nom: {exemple.nom}")
    print(f"Prénom: {exemple.prenom}")
    print(f"Classe: {exemple.classe}")
    print(f"Matières extraites: {len(exemple.matieres)}")
    for m in exemple.matieres[:3]:
        print(f"  - {m.nom}: {m.moyenne_eleve}")
else:
    print(f"Erreur ou pas de résultat: {exemple}")

Nom: None
Prénom: None
Classe: None
Matières extraites: 12
  - Anglais LV1: 15.21
  - Arts Plastiques: 15.0
  - EPS: 8.0


## 2. Mistral OCR

In [5]:
import base64

from mistralai import Mistral

# Init client
mistral_api_key = os.getenv("MISTRAL_OCR_API_KEY")
if not mistral_api_key:
    raise ValueError("MISTRAL_OCR_API_KEY non configurée dans .env")

client = Mistral(api_key=mistral_api_key)
print("Client Mistral initialisé")

Client Mistral initialisé


In [6]:
def extract_with_mistral_ocr(pdf_path: Path) -> dict:
    """Extrait le contenu d'un PDF avec Mistral OCR.
    
    Utilise l'API OCR de Mistral (mistral-ocr-2503).
    Ref: https://docs.mistral.ai/capabilities/document/
    """
    # Encoder le PDF en base64
    with open(pdf_path, "rb") as f:
        pdf_base64 = base64.standard_b64encode(f.read()).decode("utf-8")
    
    # Appel API OCR
    response = client.ocr.process(
        model="mistral-ocr-2503",
        document={
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{pdf_base64}"
        }
    )
    
    return response

In [7]:
# Test sur un seul PDF d'abord
test_pdf = RAW_DIR / "ELEVE_A.pdf"

start = time.perf_counter()
result = extract_with_mistral_ocr(test_pdf)
elapsed = time.perf_counter() - start

print(f"Temps: {elapsed:.2f}s")
print(f"Type réponse: {type(result)}")
print(f"\nRéponse:")
print(result)

Temps: 2.73s
Type réponse: <class 'mistralai.models.ocrresponse.OCRResponse'>

Réponse:
pages=[OCRPageObject(index=0, markdown="# College Test BULLETIN SCOLAIRE \n\nÉlève : ELEVE_A<br>Genre : Fille<br>Absences : 4 demi-journées (justifiées)<br>Engagements : Déléguée titulaire\n\n| Matière | Élève / <br> Classe | Appréciation |\n| :--: | :--: | :--: |\n| Anglais LV1 | 15.21 / 10.83 | Bons résultats. ELEVE_A fournit un travail régulier et sérieux à la maison tout comme en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi! |\n| Arts Plastiques | 15.00 / 14.92 | Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée. |\n| EPS | 8.00 / 12.79 | Bilan très insuffisant, ELEVE_A n'a pas réussi à s'orienter avec efficacité. Elle a beaucoup marché et s'est dispersée avec son groupe. Son travail a manqué de sérieux et d'implication dans les défis à relever en course d'orientation. De sincères efforts sont attendus au prochain trimestre. |\n| Éducation Musi

In [8]:
# Extraire tous les PDFs avec Mistral OCR
mistral_results = {}
mistral_times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem
    
    start = time.perf_counter()
    try:
        result = extract_with_mistral_ocr(pdf_path)
        mistral_results[eleve_id] = result
    except Exception as e:
        mistral_results[eleve_id] = f"ERROR: {e}"
    mistral_times[eleve_id] = time.perf_counter() - start
    
    print(f"{eleve_id}: {mistral_times[eleve_id]:.2f}s")

ELEVE_A: 2.46s
ELEVE_B: 2.70s
ELEVE_C: 2.30s
ELEVE_D: 2.59s


## 3. Comparaison

In [9]:
# Tableau comparatif des temps
import pandas as pd

comparison = []
for eleve_id in gt_by_eleve.keys():
    comparison.append({
        "eleve_id": eleve_id,
        "pdfplumber_time": pdfplumber_times.get(eleve_id, None),
        "mistral_time": mistral_times.get(eleve_id, None),
    })

df_times = pd.DataFrame(comparison)
df_times["speedup"] = df_times["mistral_time"] / df_times["pdfplumber_time"]
print("Temps d'exécution (secondes):")
print(df_times.to_string(index=False))
print(f"\nMoyenne pdfplumber: {df_times['pdfplumber_time'].mean():.2f}s")
print(f"Moyenne Mistral OCR: {df_times['mistral_time'].mean():.2f}s")

Temps d'exécution (secondes):
eleve_id  pdfplumber_time  mistral_time   speedup
 ELEVE_A         0.068749      2.455649 35.718997
 ELEVE_B         0.065140      2.701806 41.476981
 ELEVE_C         0.057187      2.298217 40.187477
 ELEVE_D         0.068517      2.589566 37.794341

Moyenne pdfplumber: 0.06s
Moyenne Mistral OCR: 2.51s


In [10]:
# Comparer le contenu extrait pour ELEVE_A
eleve_id = "ELEVE_A"
gt = gt_by_eleve[eleve_id]

print("=" * 60)
print(f"GROUND TRUTH - {eleve_id}")
print("=" * 60)
print(f"Genre: {gt['genre']}")
print(f"Absences: {gt['absences_demi_journees']}")
print(f"Matières: {len(gt['matieres'])}")
for m in gt["matieres"][:3]:
    print(f"  - {m['nom']}: {m['moyenne_eleve']} (classe: {m['moyenne_classe']})")

GROUND TRUTH - ELEVE_A
Genre: F
Absences: 4
Matières: 12
  - ANGLAIS LV1: 15.21 (classe: 10.83)
  - ARTS PLASTIQUES: 15.0 (classe: 14.92)
  - ED.PHYSIQUE & SPORT: 8.0 (classe: 12.79)


In [11]:
# Afficher le texte brut extrait par Mistral OCR (complet)
print("=" * 60)
print(f"MISTRAL OCR - {eleve_id}")
print("=" * 60)
mistral_result = mistral_results.get(eleve_id)
if mistral_result and not isinstance(mistral_result, str):
    if hasattr(mistral_result, 'pages'):
        for i, page in enumerate(mistral_result.pages):
            print(f"\n--- Page {i+1} ---")
            print(page.markdown)  # Afficher tout le contenu
    else:
        print(mistral_result)
else:
    print(f"Erreur: {mistral_result}")

MISTRAL OCR - ELEVE_A

--- Page 1 ---
# College Test BULLETIN SCOLAIRE 

Élève : ELEVE_A<br>Genre : Fille<br>Absences : 4 demi-journées (justifiées)<br>Engagements : Déléguée titulaire

| Matière | Élève / <br> Classe | Appréciation |
| :--: | :--: | :--: |
| Anglais LV1 | 15.21 / 10.83 | Bons résultats. ELEVE_A fournit un travail régulier et sérieux à la maison tout comme en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi! |
| Arts Plastiques | 15.00 / 14.92 | Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée. |
| EPS | 8.00 / 12.79 | Bilan très insuffisant, ELEVE_A n'a pas réussi à s'orienter avec efficacité. Elle a beaucoup marché et s'est dispersée avec son groupe. Son travail a manqué de sérieux et d'implication dans les défis à relever en course d'orientation. De sincères efforts sont attendus au prochain trimestre. |
| Éducation Musicale | 13.00 / 9.53 | C'est très bien quand vous voulez. Votre investissement doit être régulier. |

## 4. Comparaison cellule par cellule

Parser le markdown de Mistral OCR et comparer avec le ground truth.

In [12]:
import re


def parse_mistral_markdown(markdown: str) -> dict:
    """Parse le markdown de Mistral OCR en structure de données."""
    result = {
        "eleve": None,
        "genre": None,
        "absences": None,
        "engagements": None,
        "matieres": [],
        "moyenne_generale": None,
    }
    
    # Extraire les métadonnées (format: "Élève : ELEVE_A<br>Genre : Fille...")
    eleve_match = re.search(r"Élève\s*:\s*(\w+)", markdown)
    if eleve_match:
        result["eleve"] = eleve_match.group(1)
    
    genre_match = re.search(r"Genre\s*:\s*(\w+)", markdown)
    if genre_match:
        result["genre"] = genre_match.group(1)
    
    absences_match = re.search(r"Absences\s*:\s*(\d+)", markdown)
    if absences_match:
        result["absences"] = int(absences_match.group(1))
    
    engagements_match = re.search(r"Engagements\s*:\s*([^<\n]+)", markdown)
    if engagements_match:
        result["engagements"] = engagements_match.group(1).strip()
    
    # Extraire les lignes du tableau markdown
    # Format: | Matière | Élève / Classe | Appréciation |
    table_pattern = r"\|\s*([^|]+)\s*\|\s*([^|]+)\s*\|\s*([^|]*)\s*\|"
    rows = re.findall(table_pattern, markdown)
    
    for row in rows:
        matiere, notes, appreciation = [cell.strip() for cell in row]
        
        # Skip header rows
        if matiere in ("Matière", ":--:") or "Élève" in matiere:
            continue
        
        # Parser les notes (format: "15.21 / 10.83")
        notes_match = re.search(r"([\d.]+)\s*/\s*([\d.]+)", notes)
        moyenne_eleve = float(notes_match.group(1)) if notes_match else None
        moyenne_classe = float(notes_match.group(2)) if notes_match else None
        
        result["matieres"].append({
            "nom": matiere,
            "moyenne_eleve": moyenne_eleve,
            "moyenne_classe": moyenne_classe,
            "appreciation": appreciation,
        })
    
    # Moyenne générale
    moy_match = re.search(r"Moyenne générale\s*:\s*([\d.]+)", markdown)
    if moy_match:
        result["moyenne_generale"] = float(moy_match.group(1))
    
    return result


# Test du parser
test_markdown = mistral_results["ELEVE_A"].pages[0].markdown
parsed = parse_mistral_markdown(test_markdown)

print(f"Élève: {parsed['eleve']}")
print(f"Genre: {parsed['genre']}")
print(f"Absences: {parsed['absences']}")
print(f"Engagements: {parsed['engagements']}")
print(f"Moyenne générale: {parsed['moyenne_generale']}")
print(f"Matières: {len(parsed['matieres'])}")

Élève: ELEVE_A
Genre: Fille
Absences: 4
Engagements: Déléguée titulaire
Moyenne générale: 14.13
Matières: 12


In [13]:
# Mapping des noms de matières (PDF simplifié vs Ground Truth)
MATIERE_MAPPING = {
    "Anglais LV1": "ANGLAIS LV1",
    "Arts Plastiques": "ARTS PLASTIQUES",
    "EPS": "ED.PHYSIQUE & SPORT",
    "Éducation Musicale": "EDUCATION MUSICALE",
    "Espagnol LV2": "ESPAGNOL LV2",
    "Français": "FRANCAIS",
    "Histoire-Géographie-EMC": "HISTOIRE-GEOGRAPHIE-EMC",
    "Latin et Grec": "LCA LATIN ET GREC",
    "Mathématiques": "MATHEMATIQUES",
    "Physique-Chimie": "PHYSIQUE-CHIMIE",
    "SVT": "SCIENCES VIE & TERRE",
    "Technologie": "TECHNOLOGIE",
    "Italien LV2": "ITALIEN LV2",
}


def compare_with_ground_truth(parsed: dict, gt: dict) -> pd.DataFrame:
    """Compare les données parsées avec le ground truth."""
    comparisons = []
    
    # Index GT par nom de matière
    gt_matieres = {m["nom"]: m for m in gt["matieres"]}
    
    for m in parsed["matieres"]:
        nom_ocr = m["nom"]
        nom_gt = MATIERE_MAPPING.get(nom_ocr, nom_ocr.upper())
        gt_matiere = gt_matieres.get(nom_gt, {})
        
        # Comparer moyenne élève
        ocr_moy = m["moyenne_eleve"]
        gt_moy = gt_matiere.get("moyenne_eleve")
        moy_match = ocr_moy == gt_moy if ocr_moy and gt_moy else None
        
        # Comparer moyenne classe
        ocr_classe = m["moyenne_classe"]
        gt_classe = gt_matiere.get("moyenne_classe")
        classe_match = ocr_classe == gt_classe if ocr_classe and gt_classe else None
        
        # Comparer appréciation (normaliser les espaces)
        ocr_app = " ".join(m["appreciation"].split()) if m["appreciation"] else ""
        gt_app = " ".join(gt_matiere.get("appreciation", "").split())
        app_match = ocr_app == gt_app
        
        comparisons.append({
            "matiere_ocr": nom_ocr,
            "matiere_gt": nom_gt,
            "moy_ocr": ocr_moy,
            "moy_gt": gt_moy,
            "moy_ok": "✅" if moy_match else "❌" if moy_match is False else "⚠️",
            "classe_ocr": ocr_classe,
            "classe_gt": gt_classe,
            "classe_ok": "✅" if classe_match else "❌" if classe_match is False else "⚠️",
            "app_ok": "✅" if app_match else "❌",
            "app_len_ocr": len(ocr_app),
            "app_len_gt": len(gt_app),
        })
    
    return pd.DataFrame(comparisons)


# Comparer ELEVE_A
df_comparison = compare_with_ground_truth(parsed, gt_by_eleve["ELEVE_A"])
print("Comparaison Mistral OCR vs Ground Truth pour ELEVE_A:\n")
print(df_comparison[["matiere_ocr", "moy_ocr", "moy_gt", "moy_ok", "classe_ok", "app_ok"]].to_string(index=False))

Comparaison Mistral OCR vs Ground Truth pour ELEVE_A:

            matiere_ocr  moy_ocr  moy_gt moy_ok classe_ok app_ok
            Anglais LV1    15.21   15.21      ✅         ✅      ✅
        Arts Plastiques    15.00   15.00      ✅         ✅      ✅
                    EPS     8.00    8.00      ✅         ✅      ✅
     Éducation Musicale    13.00   13.00      ✅         ✅      ✅
           Espagnol LV2    13.83   13.83      ✅         ✅      ✅
               Français    13.21   13.21      ✅         ✅      ✅
Histoire-Géographie-EMC    14.66   14.66      ✅         ✅      ✅
          Latin et Grec    18.21   18.21      ✅         ✅      ✅
          Mathématiques    16.07   16.07      ✅         ✅      ✅
        Physique-Chimie    15.71   15.71      ✅         ✅      ✅
                    SVT    10.86   10.86      ✅         ✅      ✅
            Technologie    15.80   15.80      ✅         ✅      ✅


In [14]:
# Comparer tous les élèves
all_comparisons = []

for eleve_id in gt_by_eleve.keys():
    mistral_result = mistral_results.get(eleve_id)
    if mistral_result and not isinstance(mistral_result, str):
        parsed = parse_mistral_markdown(mistral_result.pages[0].markdown)
        df = compare_with_ground_truth(parsed, gt_by_eleve[eleve_id])
        df["eleve_id"] = eleve_id
        all_comparisons.append(df)

df_all = pd.concat(all_comparisons, ignore_index=True)

# Statistiques globales
total = len(df_all)
moy_ok = (df_all["moy_ok"] == "✅").sum()
classe_ok = (df_all["classe_ok"] == "✅").sum()
app_ok = (df_all["app_ok"] == "✅").sum()

print("=" * 60)
print("STATISTIQUES GLOBALES - Mistral OCR vs Ground Truth")
print("=" * 60)
print(f"\nTotal matières analysées: {total}")
print(f"Moyennes élève correctes: {moy_ok}/{total} ({100*moy_ok/total:.1f}%)")
print(f"Moyennes classe correctes: {classe_ok}/{total} ({100*classe_ok/total:.1f}%)")
print(f"Appréciations identiques: {app_ok}/{total} ({100*app_ok/total:.1f}%)")

STATISTIQUES GLOBALES - Mistral OCR vs Ground Truth

Total matières analysées: 46
Moyennes élève correctes: 46/46 (100.0%)
Moyennes classe correctes: 46/46 (100.0%)
Appréciations identiques: 43/46 (93.5%)


In [15]:
# Détail des appréciations qui ne matchent pas
df_errors = df_all[df_all["app_ok"] == "❌"][["eleve_id", "matiere_ocr", "app_len_ocr", "app_len_gt"]]
print(f"Appréciations différentes ({len(df_errors)}):\n")
print(df_errors.to_string(index=False))

Appréciations différentes (3):

eleve_id             matiere_ocr  app_len_ocr  app_len_gt
 ELEVE_B             Anglais LV1          218         219
 ELEVE_B                     EPS           90          91
 ELEVE_C Histoire-Géographie-EMC           56          56


### Export des extractions pour comparaison

Sauvegarder les extractions en markdown pour faciliter la comparaison.

In [16]:
# Dossier de sortie
BENCHMARK_DIR = DATA_DIR / "processed" / "benchmark-extraction"
BENCHMARK_DIR.mkdir(parents=True, exist_ok=True)


def export_ground_truth_md(gt: dict, output_path: Path):
    """Exporte le ground truth en markdown."""
    lines = [
        f"# Ground Truth - {gt['eleve_id']}",
        "",
        f"**Genre:** {gt['genre']}",
        f"**Absences:** {gt['absences_demi_journees']} demi-journées",
        f"**Engagements:** {', '.join(gt.get('engagements', [])) or 'Aucun'}",
        "",
        "## Matières",
        "",
        "| Matière | Élève | Classe | Appréciation |",
        "|---------|-------|--------|--------------|",
    ]
    
    for m in gt["matieres"]:
        app = m.get("appreciation", "").replace("|", "\\|").replace("\n", " ")
        lines.append(f"| {m['nom']} | {m['moyenne_eleve']} | {m['moyenne_classe']} | {app} |")
    
    output_path.write_text("\n".join(lines), encoding="utf-8")


def export_mistral_ocr_md(parsed: dict, output_path: Path):
    """Exporte l'extraction Mistral OCR en markdown."""
    lines = [
        f"# Mistral OCR - {parsed['eleve']}",
        "",
        f"**Genre:** {parsed['genre']}",
        f"**Absences:** {parsed['absences']} demi-journées",
        f"**Engagements:** {parsed['engagements'] or 'Aucun'}",
        "",
        "## Matières",
        "",
        "| Matière | Élève | Classe | Appréciation |",
        "|---------|-------|--------|--------------|",
    ]
    
    for m in parsed["matieres"]:
        app = m.get("appreciation", "").replace("|", "\\|").replace("\n", " ")
        lines.append(f"| {m['nom']} | {m['moyenne_eleve']} | {m['moyenne_classe']} | {app} |")
    
    output_path.write_text("\n".join(lines), encoding="utf-8")


def export_pdfplumber_md(result, output_path: Path):
    """Exporte l'extraction pdfplumber en markdown."""
    if isinstance(result, str):
        output_path.write_text(f"# Erreur\n\n{result}", encoding="utf-8")
        return
    
    lines = [
        f"# pdfplumber - {result.nom or 'N/A'}",
        "",
        f"**Genre:** {result.genre or 'N/A'}",
        f"**Absences:** {result.absences_demi_journees or 'N/A'} demi-journées",
        f"**Engagements:** {', '.join(result.engagements) if result.engagements else 'Aucun'}",
        "",
        "## Matières",
        "",
        "| Matière | Élève | Classe | Appréciation |",
        "|---------|-------|--------|--------------|",
    ]
    
    for m in result.matieres:
        app = (m.appreciation or "").replace("|", "\\|").replace("\n", " ")
        lines.append(f"| {m.nom} | {m.moyenne_eleve} | {m.moyenne_classe} | {app} |")
    
    output_path.write_text("\n".join(lines), encoding="utf-8")


# Exporter pour tous les élèves
for eleve_id in gt_by_eleve.keys():
    # Ground truth
    export_ground_truth_md(
        gt_by_eleve[eleve_id],
        BENCHMARK_DIR / f"{eleve_id}_ground_truth.md"
    )
    
    # Mistral OCR
    mistral_result = mistral_results.get(eleve_id)
    if mistral_result and not isinstance(mistral_result, str):
        parsed = parse_mistral_markdown(mistral_result.pages[0].markdown)
        export_mistral_ocr_md(parsed, BENCHMARK_DIR / f"{eleve_id}_mistral_ocr.md")
    
    # pdfplumber
    pdf_result = pdfplumber_results.get(eleve_id)
    export_pdfplumber_md(pdf_result, BENCHMARK_DIR / f"{eleve_id}_pdfplumber.md")
    
    print(f"✓ {eleve_id} exporté")

print(f"\nFichiers exportés dans: {BENCHMARK_DIR}")

✓ ELEVE_A exporté
✓ ELEVE_B exporté
✓ ELEVE_C exporté
✓ ELEVE_D exporté

Fichiers exportés dans: c:\Users\Florent\Documents\data_science\chiron\data\processed\benchmark-extraction


In [17]:
# Exemple de différence d'appréciation (si existe)
if len(df_errors) > 0:
    sample = df_errors.iloc[0]
    eleve_id = sample["eleve_id"]
    matiere = sample["matiere_ocr"]
    
    # Récupérer les appréciations
    parsed = parse_mistral_markdown(mistral_results[eleve_id].pages[0].markdown)
    ocr_app = next((m["appreciation"] for m in parsed["matieres"] if m["nom"] == matiere), "")
    
    nom_gt = MATIERE_MAPPING.get(matiere, matiere.upper())
    gt_app = next((m["appreciation"] for m in gt_by_eleve[eleve_id]["matieres"] if m["nom"] == nom_gt), "")
    
    print(f"Exemple de différence: {eleve_id} - {matiere}\n")
    print(f"OCR ({len(ocr_app)} chars):\n{ocr_app}\n")
    print(f"GT ({len(gt_app)} chars):\n{gt_app}")
else:
    print("Toutes les appréciations sont identiques!")

Exemple de différence: ELEVE_B - Anglais LV1

OCR (218 chars):
Bons résultats à l'écrit temis par une moyenne orale faible due à un accident au dernier contrôle. Je suis sûre que ELEVE_B saura y remédier au trimestre prochain en participant davantage en classe. Je compte sur elle.

GT (219 chars):
Bons résultats à l'écrit ternis par une moyenne orale faible due à un accident au dernier contrôle. Je suis sûre que ELEVE_B saura y remédier au trimestre prochain en participant davantage en classe. Je compte sur elle.


## 5. Comparaison pdfplumber vs Ground Truth

In [18]:
def compare_pdfplumber_with_gt(pdfplumber_result, gt: dict) -> pd.DataFrame:
    """Compare les données pdfplumber avec le ground truth."""
    comparisons = []
    
    if isinstance(pdfplumber_result, str):  # Error
        return pd.DataFrame()
    
    # Index GT par nom de matière
    gt_matieres = {m["nom"]: m for m in gt["matieres"]}
    
    for m in pdfplumber_result.matieres:
        nom_pdf = m.nom
        nom_gt = MATIERE_MAPPING.get(nom_pdf, nom_pdf.upper())
        gt_matiere = gt_matieres.get(nom_gt, {})
        
        # Comparer moyenne élève
        pdf_moy = m.moyenne_eleve
        gt_moy = gt_matiere.get("moyenne_eleve")
        moy_match = pdf_moy == gt_moy if pdf_moy and gt_moy else None
        
        # Comparer moyenne classe
        pdf_classe = m.moyenne_classe
        gt_classe = gt_matiere.get("moyenne_classe")
        classe_match = pdf_classe == gt_classe if pdf_classe and gt_classe else None
        
        # Comparer appréciation
        pdf_app = " ".join(m.appreciation.split()) if m.appreciation else ""
        gt_app = " ".join(gt_matiere.get("appreciation", "").split())
        app_match = pdf_app == gt_app
        
        comparisons.append({
            "matiere_pdf": nom_pdf,
            "matiere_gt": nom_gt,
            "moy_pdf": pdf_moy,
            "moy_gt": gt_moy,
            "moy_ok": "✅" if moy_match else "❌" if moy_match is False else "⚠️",
            "classe_pdf": pdf_classe,
            "classe_gt": gt_classe,
            "classe_ok": "✅" if classe_match else "❌" if classe_match is False else "⚠️",
            "app_ok": "✅" if app_match else "❌",
            "app_len_pdf": len(pdf_app),
            "app_len_gt": len(gt_app),
        })
    
    return pd.DataFrame(comparisons)


# Comparer ELEVE_A avec pdfplumber
df_pdf_comparison = compare_pdfplumber_with_gt(pdfplumber_results["ELEVE_A"], gt_by_eleve["ELEVE_A"])
print("Comparaison pdfplumber vs Ground Truth pour ELEVE_A:\n")
print(df_pdf_comparison[["matiere_pdf", "moy_pdf", "moy_gt", "moy_ok", "classe_ok", "app_ok"]].to_string(index=False))

Comparaison pdfplumber vs Ground Truth pour ELEVE_A:

            matiere_pdf  moy_pdf  moy_gt moy_ok classe_ok app_ok
            Anglais LV1    15.21   15.21      ✅         ✅      ❌
        Arts Plastiques    15.00   15.00      ✅         ✅      ❌
                    EPS     8.00    8.00      ✅         ✅      ❌
     Éducation Musicale    13.00   13.00      ✅         ✅      ❌
           Espagnol LV2    13.83   13.83      ✅         ✅      ✅
               Français    13.21   13.21      ✅         ✅      ❌
Histoire-Géographie-EMC    14.66   14.66      ✅         ✅      ❌
          Latin et Grec    18.21   18.21      ✅         ✅      ❌
          Mathématiques    16.07   16.07      ✅         ✅      ❌
        Physique-Chimie    15.71   15.71      ✅         ✅      ❌
                    SVT    10.86   10.86      ✅         ✅      ❌
            Technologie    15.80   15.80      ✅         ✅      ❌


In [19]:
# Statistiques globales pdfplumber
all_pdf_comparisons = []

for eleve_id in gt_by_eleve.keys():
    pdf_result = pdfplumber_results.get(eleve_id)
    if pdf_result and not isinstance(pdf_result, str):
        df = compare_pdfplumber_with_gt(pdf_result, gt_by_eleve[eleve_id])
        df["eleve_id"] = eleve_id
        all_pdf_comparisons.append(df)

df_all_pdf = pd.concat(all_pdf_comparisons, ignore_index=True)

total_pdf = len(df_all_pdf)
moy_ok_pdf = (df_all_pdf["moy_ok"] == "✅").sum()
classe_ok_pdf = (df_all_pdf["classe_ok"] == "✅").sum()
app_ok_pdf = (df_all_pdf["app_ok"] == "✅").sum()

print("=" * 60)
print("STATISTIQUES GLOBALES - pdfplumber vs Ground Truth")
print("=" * 60)
print(f"\nTotal matières analysées: {total_pdf}")
print(f"Moyennes élève correctes: {moy_ok_pdf}/{total_pdf} ({100*moy_ok_pdf/total_pdf:.1f}%)")
print(f"Moyennes classe correctes: {classe_ok_pdf}/{total_pdf} ({100*classe_ok_pdf/total_pdf:.1f}%)")
print(f"Appréciations identiques: {app_ok_pdf}/{total_pdf} ({100*app_ok_pdf/total_pdf:.1f}%)")

STATISTIQUES GLOBALES - pdfplumber vs Ground Truth

Total matières analysées: 46
Moyennes élève correctes: 46/46 (100.0%)
Moyennes classe correctes: 46/46 (100.0%)
Appréciations identiques: 3/46 (6.5%)


## 6. Résumé comparatif

In [20]:
# Résumé comparatif final
print("=" * 70)
print("RÉSUMÉ BENCHMARK - Mistral OCR vs pdfplumber")
print("=" * 70)

summary = pd.DataFrame({
    "Métrique": [
        "Temps moyen (s)",
        "Moyennes élève correctes",
        "Moyennes classe correctes", 
        "Appréciations identiques",
    ],
    "pdfplumber": [
        f"{df_times['pdfplumber_time'].mean():.2f}",
        f"{100*moy_ok_pdf/total_pdf:.1f}%",
        f"{100*classe_ok_pdf/total_pdf:.1f}%",
        f"{100*app_ok_pdf/total_pdf:.1f}%",
    ],
    "Mistral OCR": [
        f"{df_times['mistral_time'].mean():.2f}",
        f"{100*moy_ok/total:.1f}%",
        f"{100*classe_ok/total:.1f}%",
        f"{100*app_ok/total:.1f}%",
    ],
})

print(summary.to_string(index=False))

print("\n" + "=" * 70)
print("CONCLUSION")
print("=" * 70)
print("""
- Mistral OCR est ~30x plus lent mais retourne du markdown structuré
- Les deux méthodes extraient correctement les notes numériques
- Vérifier les différences d'appréciations (ponctuation, espaces)
- Mistral OCR ne nécessite pas de logique de parsing de tableaux spécifique
""")

RÉSUMÉ BENCHMARK - Mistral OCR vs pdfplumber
                 Métrique pdfplumber Mistral OCR
          Temps moyen (s)       0.06        2.51
 Moyennes élève correctes     100.0%      100.0%
Moyennes classe correctes     100.0%      100.0%
 Appréciations identiques       6.5%       93.5%

CONCLUSION

- Mistral OCR est ~30x plus lent mais retourne du markdown structuré
- Les deux méthodes extraient correctement les notes numériques
- Vérifier les différences d'appréciations (ponctuation, espaces)
- Mistral OCR ne nécessite pas de logique de parsing de tableaux spécifique

